In [1]:
import arcpy
import pandas as pd
import psycopg2
from psycopg2 import sql
import numpy as np
import os
import folium

In [2]:
cwd = os.getcwd() # This is a global variable for where the notebook is (must change if running in arcpro)

# Make it workspace

arcpy.env.workspace = os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')

arcpy.env.overwriteOutput = True # Overwrite layers is okay

save_path_csv = os.path.join(cwd, '..', '..', 'data')
save_path_gdb = os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')
historic_path = os.path.join(cwd, '..', '..', 'data', 'purpleair_historic.csv')
station_path = os.path.join(cwd, '..', '..', 'data', 'purpleair_stations.csv')
boundary_path = os.path.join(cwd, '..', '..', 'data', 'mpls_boundary.geojson')

In [3]:
output_fc = 'mpls_boundary'
output_fc_path = os.path.join(save_path_gdb, output_fc)

# Convert the GeoJSON file to a feature class
arcpy.JSONToFeatures_conversion(boundary_path, output_fc)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\mpls_boundary'>

In [4]:
df = pd.read_csv(historic_path)

# Convert the 'timestamp' column to a datetime object
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Filter the rows that have a timestamp of 4/1/23
date_filter = df['timestamp'].dt.date == pd.to_datetime('4/1/23').date()
filtered_df = df[date_filter]

filtered_df.to_csv('historic_april1.csv', index=False)
historic_filtered_path = os.path.join(cwd, '..', '..', 'data', 'historic_april1.csv')

In [5]:
# Load the first CSV file into a DataFrame
df1 = pd.read_csv(station_path)

# Load the second CSV file into a DataFrame
df2 = pd.read_csv(historic_filtered_path)

# Merge the two DataFrames together on a common column
merged_df = pd.merge(df1, df2, on='sensor_index')

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('merged_file.csv', index=False)
merged_path = os.path.join(cwd, '..', '..', 'data', 'merged_file.csv')

In [6]:
# Set output point feature class name and path
output_point_fc = "stations_XYTableToPoint"
output_point_path = os.path.join(save_path_gdb, output_point_fc)

# Specify x,y fields
x_field = "longitude"
y_field = "latitude"

# Use XYTableToPoint tool to create point features
station_points = arcpy.management.XYTableToPoint(merged_path, output_point_path, x_field, y_field)

In [7]:
'''
output_table = "joined_table"
arcpy.management.AddJoin(
    in_layer_or_view=station_points,
    in_field="sensor_index",
    join_table=historic_filtered_path,
    join_field="sensor_index",
    join_type="KEEP_ALL",
    index_join_fields="NO_INDEX_JOIN_FIELDS"
)

# Export the joined layer to a new table
historic_apr1 = arcpy.management.CopyRows("stations_XYTableToPoint", output_table)
'''

'\noutput_table = "joined_table"\narcpy.management.AddJoin(\n    in_layer_or_view=station_points,\n    in_field="sensor_index",\n    join_table=historic_filtered_path,\n    join_field="sensor_index",\n    join_type="KEEP_ALL",\n    index_join_fields="NO_INDEX_JOIN_FIELDS"\n)\n\n# Export the joined layer to a new table\nhistoric_apr1 = arcpy.management.CopyRows("stations_XYTableToPoint", output_table)\n'

## PM 2.5 Interpolation

### Kriging PM 2.5

In [29]:
out_kriging_raster = "Kriging_Apr1"
arcpy.ga.EmpiricalBayesianKriging(
    in_features="stations_XYTableToPoint",
    z_field="pm2_5",
    out_ga_layer=None,
    out_raster=out_kriging_raster,
    cell_size=0.000932919999999967,
    transformation_type="NONE",
    max_local_points=100,
    overlap_factor=1,
    number_semivariograms=100,
    search_neighborhood="NBRTYPE=StandardCircular RADIUS=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    output_type="PREDICTION",
    quantile_value=0.5,
    threshold_type="EXCEED",
    probability_threshold=None,
    semivariogram_model_type="POWER"
)

<Result ''>

### Clip Kriging

In [30]:
out_clip_kriging_raster = "Kriging_clip_Apr1"
arcpy.management.Clip(
    in_raster=out_kriging_raster,
    rectangle="-93.32910837 44.8905885090001 -93.194328522 45.0512462900001",
    out_raster=out_clip_kriging_raster,
    in_template_dataset='mpls_boundary',
    nodata_value="3.4e+38",
    clipping_geometry="NONE",
    maintain_clipping_extent="NO_MAINTAIN_EXTENT"
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\Kriging_clip_Apr1'>

### IDW PM 2.5

In [25]:
out_idw_raster = "IDW_Apr1"
arcpy.ga.IDW(
    in_features="stations_XYTableToPoint",
    z_field="pm2_5",
    out_ga_layer=None,
    out_raster=out_idw_raster,
    cell_size=10,
    power=2,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=9.87111293434023E-02 S_MINOR=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    weight_field=None
)

<Result ''>

### Clip IDW

In [11]:
out_clip_idw_raster = "IDW_clip_Apr1"
arcpy.management.Clip(
    in_raster=out_idw_raster,
    rectangle="-93.32910837 44.8905885090001 -93.194328522 45.0512462900001",
    out_raster=out_clip_idw_raster,
    in_template_dataset='mpls_boundary',
    nodata_value="3.4e+38",
    clipping_geometry="NONE",
    maintain_clipping_extent="NO_MAINTAIN_EXTENT"
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\IDW_clip_Apr1'>

### Local Polynomial PM 2.5

In [33]:
out_poly_raster = "Poly_Apr1"
arcpy.ga.LocalPolynomialInterpolation(
    in_features="stations_XYTableToPoint",
    z_field="pm2_5",
    out_ga_layer=None,
    out_raster=out_poly_raster,
    cell_size=0.000932919999999967,
    power=1,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=9.87111293434023E-02 S_MINOR=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    kernel_function="EXPONENTIAL",
    bandwidth=None,
    use_condition_number="NO_USE_CONDITION_NUMBER",
    condition_number=None,
    weight_field=None,
    output_type="PREDICTION"
)

<Result ''>

### Clip Local Polynomail

In [34]:
out_clip_poly_raster = "Poly_clip_Apr1"
arcpy.management.Clip(
    in_raster=out_poly_raster,
    rectangle="-93.32910837 44.8905885090001 -93.194328522 45.0512462900001",
    out_raster=out_clip_poly_raster,
    in_template_dataset='mpls_boundary',
    nodata_value="3.4e+38",
    clipping_geometry="NONE",
    maintain_clipping_extent="NO_MAINTAIN_EXTENT"
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\Poly_clip_Apr1'>

## Temperature Interpolation

### Kriging Temperature

In [31]:
out_kriging_temp_raster = "Kriging_Temp_Apr1"
arcpy.ga.EmpiricalBayesianKriging(
    in_features="stations_XYTableToPoint",
    z_field="temperature",
    out_ga_layer=None,
    out_raster=out_kriging_temp_raster,
    cell_size=0.000932919999999967,
    transformation_type="NONE",
    max_local_points=100,
    overlap_factor=1,
    number_semivariograms=100,
    search_neighborhood="NBRTYPE=StandardCircular RADIUS=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    output_type="PREDICTION",
    quantile_value=0.5,
    threshold_type="EXCEED",
    probability_threshold=None,
    semivariogram_model_type="POWER"
)

<Result ''>

### Clip Kriging Temp

In [32]:
out_clip_kriging_temp_raster = "Kriging_clip_temp_Apr1"
arcpy.management.Clip(
    in_raster=out_kriging_temp_raster,
    rectangle="-93.32910837 44.8905885090001 -93.194328522 45.0512462900001",
    out_raster=out_clip_kriging_temp_raster,
    in_template_dataset='mpls_boundary',
    nodata_value="3.4e+38",
    clipping_geometry="NONE",
    maintain_clipping_extent="NO_MAINTAIN_EXTENT"
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\Kriging_clip_temp_Apr1'>

### IDW Temp

In [16]:
out_idw_temp_raster = "IDW_Temp_Apr1"
arcpy.ga.IDW(
    in_features="stations_XYTableToPoint",
    z_field="temperature",
    out_ga_layer=None,
    out_raster=out_idw_temp_raster,
    cell_size=0.000932919999999967,
    power=2,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=9.87111293434023E-02 S_MINOR=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    weight_field=None
)

<Result ''>

### Clip IDW

In [17]:
out_clip_idw_temp_raster = "IDW_clip_temp_Apr1"
arcpy.management.Clip(
    in_raster=out_idw_temp_raster,
    rectangle="-93.32910837 44.8905885090001 -93.194328522 45.0512462900001",
    out_raster=out_clip_idw_temp_raster,
    in_template_dataset='mpls_boundary',
    nodata_value="3.4e+38",
    clipping_geometry="NONE",
    maintain_clipping_extent="NO_MAINTAIN_EXTENT"
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\IDW_clip_temp_Apr1'>

### Local Polynomial Temp

In [35]:
out_poly_temp_raster = "Poly_temp_Apr1"
arcpy.ga.LocalPolynomialInterpolation(
    in_features="stations_XYTableToPoint",
    z_field="temperature",
    out_ga_layer=None,
    out_raster=out_poly_temp_raster,
    cell_size=0.000932919999999967,
    power=1,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=9.87111293434023E-02 S_MINOR=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    kernel_function="EXPONENTIAL",
    bandwidth=None,
    use_condition_number="NO_USE_CONDITION_NUMBER",
    condition_number=None,
    weight_field=None,
    output_type="PREDICTION"
)

<Result ''>

### Clip Poly

In [37]:
out_clip_poly_temp_raster = "Poly_clip_temp_Apr1"
arcpy.management.Clip(
    in_raster=out_poly_temp_raster,
    rectangle="-93.32910837 44.8905885090001 -93.194328522 45.0512462900001",
    out_raster=out_clip_poly_temp_raster,
    in_template_dataset='mpls_boundary',
    nodata_value="3.4e+38",
    clipping_geometry="NONE",
    maintain_clipping_extent="NO_MAINTAIN_EXTENT"
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\Poly_clip_temp_Apr1'>

## Cross Validation

### Kriging PM 2.5

### IDW PM 2.5

### Local Polynomial PM 2.5

### Kriging Temperature

### IDW Temprature

### Local Polynomial Temperature